# Jetpack Compose - DataStore - Podstawy

W tej aplikacji przyjrzymy się zastosowaniu **DataStore** w celu zapisania niewielkich ilości danych w pliku.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExMHZ0Mmt6cjJwYjVrYzE0NWdpbHNqYXM5ZjVvNzc1bmhqZm1zY2F5NSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/kd4nprniMKNt9fBZIv/giphy.gif" width="200" />

Jest to biblioteka, która pomaga w przechowywaniu i zarządzaniu danymi w sposób bardziej zoptymalizowany i bezpieczny niż starsze rozwiązania, takie jak `SharedPreferences` (zostanie omówiony w kolejnym przykładzie).

`DataStore` jest często wykorzystywany do przechowywania niewielkich ilości danych, takich jak preferencje użytkownika, ustawienia aplikacji, stan sesji i tym podobne. Nie jest on jednak odpowiedni do przechowywania dużej ilości danych, jakie można znaleźć w bazach danych.

Oferuje dwa główne rodzaje implementacji:
- `Preferences DataStore` - Ten rodzaj jest podobny do `SharedPreferences`, ale jest oparty na protokole `Kotlin Coroutines` i zapewnia bezpieczne przechowywanie danych. Można w nim przechowywać dane w postaci *klucz-wartość*, gdzie klucze są ciągami znaków, a wartości mogą być różnymi typami danych, takimi jak liczby, ciągi znaków itp.
- `Proto DataStore` - Ten rodzaj pozwala na zapisywanie danych w formacie `protobuf`. Jest to format serializacji danych opracowany przez firmę Google.

Główne cehy korzystania z `DataStore` to:
- Wsparcie dla asynchronicznych operacji dzięki `Kotlin Coroutines`.
- Bezpieczeństwo wątkowe - unika problemów synchronizacyjnych, które mogą występować w przypadku `SharedPreferences`.
- Automatyczne obsługiwania zmian wartości, co pozwala na łatwe reagowanie na zmiany danych.
- Możliwość zapisywania i odczytywania niestandardowych typów danych dzięki `Protobuf`.

W przykładowej aplikacji wykorzystamy `DataStore` do zapisania i odczytania nazwy użytkownika.

Dodajmy niezbędne zależności do projektu:

do bloku `dependencies`
```kotlin
    implementation (libs.androidx.lifecycle.viewmodel.compose)
    implementation (libs.androidx.lifecycle.viewmodel.ktx)
    implementation (libs.androidx.lifecycle.runtime.compose)
    implementation (libs.androidx.datastore.preferences)
```

do pliku `libs.versions.toml`

```kotlin
[versions]
agp = "8.5.2"
datastorePreferences = "1.1.1"
kotlin = "1.9.0"
coreKtx = "1.13.1"
junit = "4.13.2"
junitVersion = "1.2.1"
espressoCore = "3.6.1"
lifecycleRuntimeKtx = "2.8.4"
activityCompose = "1.9.1"
composeBom = "2024.04.01"

[libraries]
androidx-core-ktx = { group = "androidx.core", name = "core-ktx", version.ref = "coreKtx" }
androidx-datastore-preferences = { module = "androidx.datastore:datastore-preferences", version.ref = "datastorePreferences" }
androidx-lifecycle-runtime-compose = { module = "androidx.lifecycle:lifecycle-runtime-compose", version.ref = "lifecycleRuntimeKtx" }
androidx-lifecycle-viewmodel-compose = { module = "androidx.lifecycle:lifecycle-viewmodel-compose", version.ref = "lifecycleRuntimeKtx" }
androidx-lifecycle-viewmodel-ktx = { module = "androidx.lifecycle:lifecycle-viewmodel-ktx", version.ref = "lifecycleRuntimeKtx" }
junit = { group = "junit", name = "junit", version.ref = "junit" }
androidx-junit = { group = "androidx.test.ext", name = "junit", version.ref = "junitVersion" }
androidx-espresso-core = { group = "androidx.test.espresso", name = "espresso-core", version.ref = "espressoCore" }
androidx-lifecycle-runtime-ktx = { group = "androidx.lifecycle", name = "lifecycle-runtime-ktx", version.ref = "lifecycleRuntimeKtx" }
androidx-activity-compose = { group = "androidx.activity", name = "activity-compose", version.ref = "activityCompose" }
androidx-compose-bom = { group = "androidx.compose", name = "compose-bom", version.ref = "composeBom" }
androidx-ui = { group = "androidx.compose.ui", name = "ui" }
androidx-ui-graphics = { group = "androidx.compose.ui", name = "ui-graphics" }
androidx-ui-tooling = { group = "androidx.compose.ui", name = "ui-tooling" }
androidx-ui-tooling-preview = { group = "androidx.compose.ui", name = "ui-tooling-preview" }
androidx-ui-test-manifest = { group = "androidx.compose.ui", name = "ui-test-manifest" }
androidx-ui-test-junit4 = { group = "androidx.compose.ui", name = "ui-test-junit4" }
androidx-material3 = { group = "androidx.compose.material3", name = "material3" }

[plugins]
android-application = { id = "com.android.application", version.ref = "agp" }
jetbrains-kotlin-android = { id = "org.jetbrains.kotlin.android", version.ref = "kotlin" }




```

Chcemy dodawać nazwy użytkowników, podobnie jak w poprzednich przykładach dodajmy *dummydata* w celu ułatwienia generacji przykładowych nazw.

In [ ]:
object DataProvider {

    private val usernames = listOf(
        "CoolDude123",
        "SuperStar",
        "GamerGirl99",
        "TechMaster",
        "MusicLover",
        "FitnessFreak",
        "Traveler_21",
        "FoodieQueen",
        "NatureLover",
        "Fashionista",
        "Bookworm42",
        "MovieBuff",
        "AdventureSeeker",
        "PetLover_87",
        "SportsFanatic",
        "ArtisticSoul",
        "StarGazer",
        "YogaEnthusiast",
        "PhotographyPro",
        "DreamChaser",
        "BeachBum123",
        "CoffeeAddict",
        "GameChanger",
        "LaughOutLoud",
        "MindfulSoul",
        "HappyGoLucky",
        "TechGeek1",
        "FoodExplorer",
        "FitnessJunkie"
    )

    val username: String
        get() = usernames.random()
}

Dodajmy klasę pomocniczą `SaveUsernameDataStore`, która służy do przechowywania i pobierania nazwy użytkownika przy użyciu `DataStore`. Wykorzystamy wzorzec Singleton, ponieważ chcemy posiadać tylko jeden obiekt przez który będziemy komunikowac się z `DataStore`.

In [ ]:
object SaveUsernameDataStore {
    private val Context.dataStore: DataStore<Preferences> by preferencesDataStore("user_prefs")
    private val USERNAME_KEY = stringPreferencesKey("USERNAME")

    suspend fun storeUsername(context: Context, username: String) {
        context.dataStore.edit { preferences ->
            preferences[USERNAME_KEY] = username
        }
    }

    fun getUsernameFlow(context: Context): Flow<String> {
        return context.dataStore.data.map { preferences ->
            preferences[USERNAME_KEY] ?: ""
        }
    }
}

- `private val Context.dataStore: DataStore<Preferences> by preferencesDataStore("user_prefs")` - Ta linia definiuje rozszerzenie (extension property) dla klasy `Context`. Tworzy ono obiekt `DataStore` typu `Preferences` o nazwie `user_prefs`. Ten `DataStore` będzie używany do przechowywania i pobierania preferencji użytkownika.
- `private val USERNAME_KEY = stringPreferencesKey("USERNAME")` - Ta linia definiuje prywatną stałą, która reprezentuje klucz używany do przechowywania i pobierania nazwy użytkownika w `DataStore`. Jest to obiekt typu `Preferences.Key<String>`.
- `suspend fun storeUsername(context: Context, username: String)` Ta funkcja służy do zapisywania nazwy użytkownika w `DataStore`.
- `fun getUsernameFlow(context: Context): Flow<String>` - Ta funkcja zwraca strumień asynchroniczny typu `Flow<String>`, który będzie emitować nazwę użytkownika za każdym razem, gdy wartość ulegnie zmianie.

Dodajmy repozytorium

In [ ]:
class UserRepository(private val application: Application) {
    fun getUsername() = SaveUsernameDataStore.getUsernameFlow(application)
    suspend fun add(username: String) = SaveUsernameDataStore.storeUsername(application, username)
    suspend fun clear() = SaveUsernameDataStore.storeUsername(application, "")
}

Podobnie jak przy wykorzystaniu `Room`, musimy przekazać kontekst aplikacji jako parametr konstruktora. Więc musimy przekazać go również przez `ViewModel`, który domyślnie nie przyjmuje żadnych parametrów, aby to zmienić musimy zaimplementować własną fabrykę.

In [ ]:
class UserViewModelFactory(private val application: Application) :
    ViewModelProvider.Factory {
    override fun <T : ViewModel> create(modelClass: Class<T>): T {
        return UserViewModel(application) as T
    }
}

Dodajmy `Viewmodel` - w kodzie nie ma żadnych nowych elementów.

In [ ]:
class UserViewModel(application: Application) : ViewModel() {
    private val repository: UserRepository
    private val _username = MutableStateFlow("")
    val username: StateFlow<String>
        get() = _username

    init {
        repository = UserRepository(application)
        fetchUser()
    }

    private fun fetchUser() {
        viewModelScope.launch {
            repository.getUsername().collect { username ->
                _username.value = username
            }
        }
    }

    fun addUsername(username: String) {
        viewModelScope.launch {
            repository.add(username)
        }
    }

    fun clearUsername(){
        viewModelScope.launch {
            repository.clear()
        }
    }
}

Następnie dodajmy funkcję renderującą ekran główny aplikacji.

In [ ]:
@Composable
fun MainScreen(){

    val viewModel: UserViewModel = viewModel(
        LocalViewModelStoreOwner.current!!,
        "UserViewModel",
        UserViewModelFactory(LocalContext.current.applicationContext as Application)
    )
    val currentUsername by viewModel.username.collectAsStateWithLifecycle()

    Column(modifier = Modifier.fillMaxSize()) {

        Text(
            text = "Current saved username: $currentUsername",
            modifier = Modifier.padding(10.dp),
            textAlign = TextAlign.Center,
            fontSize = 24.sp
        )

        Button(
            onClick = { viewModel.addUsername(DataProvider.username) },
            modifier = Modifier.fillMaxWidth().padding(4.dp)
        ) {
            Text(text = "ADD")
        }

        Button(
            onClick = { viewModel.clearUsername() },
            modifier = Modifier.fillMaxWidth().padding(4.dp)
        ) {
            Text(text = "CLEAR")
        }
    }
}

W powyższym kodzie jedynym nowym elementem może być uzyskanie instancji `UserViewModel`

In [ ]:
val viewModel: UserViewModel = viewModel(
    LocalViewModelStoreOwner.current!!,
    "UserViewModel",
    UserViewModelFactory(LocalContext.current.applicationContext as Application)
)

Chcemy skorzystać z zaimplementowanej wcześniej fabryki - funkcja `viewModel` przyjmuje kilka argumentów:

- viewModelStoreOwner: ViewModelStoreOwner - Parametr określa właściciela `ViewModelStore`, do którego `ViewModel` będzie przywiązany. `ViewModelStore` to miejsce przechowywania `ViewModel` i ich stanu między rekompozycjami.
- `key: String` - Jest unikalnym kluczem, który identyfikuje `ViewModel` wewnątrz `ViewModelStore`.
- `factory: () -> T` - Tworzy instancję ViewModelu. Jest to sposób na odroczenie tworzenia `ViewModel` do momentu, gdy jest rzeczywiście potrzebny. Funkcja `factory` nie przyjmuje żadnych argumentów i zwraca instancję `ViewModel` (`T`). Gdy `viewModel()` jest wywoływane w środowisku `Compose`, funkcja `factory` zostanie wykonana tylko raz i instancja `ViewModel` będzie przechowywana i dostępna do ponownego użycia między rekompozycjami.

Przekazane parametry:
- `LocalViewModelStoreOwner.current!!` - `LocalViewModelStoreOwner` to zmienna lokalna, która dostarcza obiekt `ViewModelStoreOwner`. W tym przypadku wykorzystujemy `LocalViewModelStoreOwner.current!!`, co oznacza, że `ViewModel` jest przywiązany do bieżącego właściciela `ViewModelStoreOwner` w drzewie `Compose`.
- `"UserViewModel"` - Jest to unikatowy klucz, który identyfikuje `ViewModel` w `ViewModelStore`.
- `UserViewModelFactory(LocalContext.current.applicationContext as Application)` - Jest to instancja `UserViewModelFactory`, która dostarcza zależność (`Application`). W ten sposób `UserViewModel` może uzyskać dostęp do kontekstu aplikacji, co jest niezbędne do uzyskania instancji bazy danych.

Wywołajmy funkcję w głównej aktywności.

In [ ]:
class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContent {
            DataSoreBasicsComposeTheme {
                // A surface container using the 'background' color from the theme
                Surface(
                    modifier = Modifier.fillMaxSize(),
                    color = MaterialTheme.colorScheme.background
                ) {
                    MainScreen()
                }
            }
        }
    }
}

Możemy przetestować aplikację.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExMHZ0Mmt6cjJwYjVrYzE0NWdpbHNqYXM5ZjVvNzc1bmhqZm1zY2F5NSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/kd4nprniMKNt9fBZIv/giphy.gif" width="200" />